# 0056 回測模型
## 2007~2018
> 策略1：每年投入固定資金，不賣，下跌x%買i股 策略2：除權息後買不賣

In [10]:
# 讀取csv(資料來源XQ匯入excel調整)
#['時間(yyyy/mm/dd)', '開盤價', '最高價', '最低價', '收盤價', '漲跌幅(%)', '成交量']
import csv
temp=[]
temp_all=[[] for i in range(19)]
with open('0056.csv') as cf:
    rows = csv.reader(cf)
    for row in rows:
        temp.append(row)
        
# str->float
for i in range(len(temp)):
    for j in range(len(temp[i])):
        hh = temp[i][j]
        if type(hh)==str:
            try:
                temp[i][j]=float(hh)
            except:
                pass
            
## 以年分割資料(2007年以後)
itertemp = iter(temp)
next(itertemp)
for i in itertemp:
    year = int(i[0][0:4])-2007
    temp_all[year].append(i)
    
#除權息和金額    
dividend={
'2017/10/30':0.95,
'2016/10/26':1.3,
'2015/10/26':1,
'2014/10/24':1,
'2013/10/24':0.85,
'2012/10/24':1.3,
'2011/10/26':2.2,
'2009/10/23':2
}

In [11]:
# 測試股利
import doctest
def test_dividend():
    """
    >>> '2009/10/23' in dividend.keys()
    True
    >>> dividend["2009/10/23"]
    2
    """
doctest.testmod()

TestResults(failed=0, attempted=2)

In [12]:
def findbest(start_money,lower,lower_buy,temp):
    
    ##初始化金額、股數、flag
    money=start_money
    stock = 0
    flag=False
    
    # 如果傳的是年就合併所有資料
    if len(temp) < 30:
        flag=True
        test=[]
        for i in temp:
            test.extend(i)
        temp=test
    
    ## 從有漲跌幅的開始到結束
    count=1
    for row in temp[2:-1]:
        
        #每年投入start_money
        count+=1
        if count%250==0:
            money+=start_money
            
        #除權息
        if row[0] in dividend.keys():
#             print("===\n new money\t{}+{}股*{}元\n===".format(money,stock,dividend[row[0]]))
            money+=stock*dividend[row[0]]
           
            
                   
        #跌買,花錢
        if row[5]<-1*lower and money>lower_buy*row[4]:
            stock+=lower_buy
            money-=row[4]*lower_buy
            if flag:
                print("{}買進\t剩餘:{:.0f}元\t剩餘:{}股".format(row[0],money,stock))
        
    total = int(money+row[4]*stock)
    
    #年回測
    if flag:
        print("-------------------------------------")
        print("如果漲跌{}%,買{}股".format(lower,lower_buy))
        print("結:total={},money={},stock={}".format(total,money,stock))
        print("淨利:%d萬\t總投入%d萬"%((total-int(count/250+1)*start_money)/10000,(int(count/250+1)*start_money)/10000))
        

    #全部回測，而且只取賺的,return 總投入(萬),總金額(萬),現金(萬),股數  
    elif total > 0:
        return [int(count/250+1)*start_money/10000,total/10000,money/10000,stock]

In [13]:
print("Find best parameter")
print("===================")
##跑回側
# x=每期投入幾萬元(range)
# i*10=lowerbound
# j*100=buy
data=[]
for x in range(10,31,1):
    money=x*10000
    earn_max=[0,0,0,0]
    best_parameter=[0,0]
    for i in range(100,-1,-5):
        for j in range(0,501,10):
            try:
                earn=findbest(money,i/10,j*100,temp)
                if earn_max[1]<earn[1]:
                    earn_max=earn.copy()
                    best_parameter=[i/10,j*100]
            except:
                pass
                        
    data.append([x,best_parameter,earn_max])
    
    print("如果跌{0[0]}%,買{0[1]}股".format(best_parameter))
    print("結:total={0[1]}萬元,money={0[2]}萬元,stock={0[3]}".format(earn_max))
    print("起始資金{}萬元，總投入{}萬元".format(x,int(earn_max[0])))
    print('---------------------------------------------')

Find best parameter
如果跌0.5%,買14000股
結:total=185.082萬元,money=32.818萬元,stock=56000
起始資金10萬元，總投入110萬元
---------------------------------------------
如果跌0.5%,買15000股
結:total=204.745萬元,money=0.82萬元,stock=75000
起始資金11萬元，總投入121萬元
---------------------------------------------
如果跌0.5%,買16000股
結:total=220.688萬元,money=3.168萬元,stock=80000
起始資金12萬元，總投入132萬元
---------------------------------------------
如果跌0.5%,買10000股
結:total=240.18萬元,money=22.66萬元,stock=80000
起始資金13萬元，總投入143萬元
---------------------------------------------
如果跌0.5%,買19000股
結:total=260.077萬元,money=1.7720000000000058萬元,stock=95000
起始資金14萬元，總投入154萬元
---------------------------------------------
如果跌0.5%,買21000股
結:total=277.623萬元,money=49.227萬元,stock=84000
起始資金15萬元，總投入165萬元
---------------------------------------------
如果跌0.5%,買12000股
結:total=294.14萬元,money=0.488萬元,stock=108000
起始資金16萬元，總投入176萬元
---------------------------------------------
如果跌2.5%,買25000股
結:total=315.55萬元,money=43.65萬元,stock=100000
起始資金17萬元，總投入187萬元
---------------------

In [14]:
## 統計
from collections import Counter
test=[str(i[1]) for i in data]
counter=Counter(test)
counter

Counter({'[0.5, 10000]': 1,
         '[0.5, 12000]': 1,
         '[0.5, 14000]': 1,
         '[0.5, 15000]': 1,
         '[0.5, 16000]': 2,
         '[0.5, 19000]': 1,
         '[0.5, 20000]': 1,
         '[0.5, 21000]': 1,
         '[0.5, 25000]': 1,
         '[0.5, 26000]': 1,
         '[0.5, 28000]': 1,
         '[0.5, 30000]': 1,
         '[0.5, 32000]': 1,
         '[0.5, 34000]': 1,
         '[0.5, 37000]': 1,
         '[0.5, 38000]': 1,
         '[0.5, 40000]': 1,
         '[0.5, 41000]': 1,
         '[2.5, 25000]': 1,
         '[2.5, 35000]': 1})

In [35]:
#年回測
findbest(120000,0.5,16000,temp_all[0:-1])

2008/12/29買進	剩餘:28960元	剩餘:16000股
2011/12/19買進	剩餘:2400元	剩餘:32000股
2014/1/6買進	剩餘:66560元	剩餘:48000股
2016/1/20買進	剩餘:73120元	剩餘:64000股
2018/1/30買進	剩餘:34560元	剩餘:80000股
-------------------------------------
如果漲跌0.5%,買16000股
結:total=2209760,money=34560.0,stock=80000
淨利:88萬	總投入132萬


In [31]:
# 2010/7/6
class Test():
 
    def __init__(self,add_money):
        self.add_money = add_money
        self.stock = 0
        self.money = add_money
        self.buy=[
        [1000000000,0],#2008
        [21.76,2],#2009
        [1000000000,0],#2010
        [23.0,2.2],#2011
        [22.38,1.3],#2012
        [23.31,1],#2013
        [22.75,0.85],#2014
        [21.93,1],#2015
        [23.79,1.3],#2016
        [25.5,0.95],#2017
        [1000000000,0]#2018
        ]
    
    def go(self):
        
        for i in range(len(self.buy)):
            self.money += self.stock*self.buy[i][1]
            buystock = int(self.money/self.buy[i][0]/1000)*1000
            self.stock += buystock
            self.money = self.money-buystock*self.buy[i][0]+self.add_money
            print(self.money,self.stock)
            
            
    def total(self):
        
        print("結:total={}元,money={}元,stock={}".format(self.stock*27.23+self.money,self.money,self.stock))
        

In [32]:
test=Test(120000)
test.go()
test.total()

240000 0
120639.99999999997 11000
240639.99999999997 11000
131840.0 22000
123780.0 29000
132920.0 35000
123420.0 42000
131910.0 49000
125290.0 57000
120940.0 64000
240940.0 64000
結:total=1983660.0元,money=240940.0元,stock=64000


# 只用漲跌幅找不太出結果而且比0055爛